In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
# from utils.timefeatures import time_features
import warnings

# 데이터로더 내 일부 클래스 로드
- Dataset_ETT_minute

In [2]:
class Dataset_ETT_minute(Dataset):
    def __init__(self, root_path, flag='train', size=None,
                 features='S', data_path='ETTm1.csv',
                 target='OT', scale=True, timeenc=0, freq='t'):
        # size [seq_len, label_len, pred_len]
        # info
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path,
                                          self.data_path))

        border1s = [0, 12 * 30 * 24 * 4 - self.seq_len, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - self.seq_len]
        border2s = [12 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        df_stamp = df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            df_stamp['minute'] = df_stamp.date.apply(lambda row: row.minute, 1)
            df_stamp['minute'] = df_stamp.minute.map(lambda x: x // 15)
            data_stamp = df_stamp.drop(['date'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)

        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

# 클래스 테스트

In [3]:
# 데이터 경로 설정
root_path = './ETT-small'
data_path = 'ETTm1.csv'

In [4]:
# 원본 데이터 확인
df = pd.read_csv(os.path.join(root_path, data_path))
print("원본 데이터 shape:", df.shape)
print("\n첫 5행 데이터:")
print(df.head())
print("\n컬럼명:", df.columns.tolist())

원본 데이터 shape: (69680, 8)

첫 5행 데이터:
                  date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0  2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1  2016-07-01 00:15:00  5.760  2.076  1.492  0.426  4.264  1.401  30.459999
2  2016-07-01 00:30:00  5.760  1.942  1.492  0.391  4.234  1.310  30.038000
3  2016-07-01 00:45:00  5.760  1.942  1.492  0.426  4.234  1.310  27.013000
4  2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001

컬럼명: ['date', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']


In [5]:
df

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401,30.459999
2,2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310,30.038000
3,2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310,27.013000
4,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
...,...,...,...,...,...,...,...,...
69675,2018-06-26 18:45:00,9.310,3.550,5.437,1.670,3.868,1.462,9.567000
69676,2018-06-26 19:00:00,10.114,3.550,6.183,1.564,3.716,1.462,9.567000
69677,2018-06-26 19:15:00,10.784,3.349,7.000,1.635,3.746,1.432,9.426000
69678,2018-06-26 19:30:00,11.655,3.617,7.533,1.706,4.173,1.523,9.426000


In [7]:
# Dataset 인스턴스 생성 (train, val, test 각각)
train_dataset = Dataset_ETT_minute(
    root_path=root_path,
    data_path=data_path,
    flag='train',    
    size=[96, 48, 24],  # [seq_len, label_len, pred_len]
    features='M',  # 다변량 데이터 사용
    target='OT'  # 목표변수
)

val_dataset = Dataset_ETT_minute(
    root_path=root_path,
    data_path=data_path,
    flag='val',
    scale='False',
    size=[96, 48, 24],
    features='M',
    target='OT'
)

test_dataset = Dataset_ETT_minute(
    root_path=root_path,
    data_path=data_path,
    flag='test',
    scale='False',
    size=[96, 48, 24],
    features='M',
    target='OT'
)

# 데이터셋 크기 확인
print("\n데이터셋 크기:")
print(f"Train: {len(train_dataset)}")
print(f"Validation: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")

C:\Users\User\AppData\Local\Temp\ipykernel_3336\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values
C:\Users\User\AppData\Local\Temp\ipykernel_3336\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values



데이터셋 크기:
Train: 34441
Validation: 11497
Test: 11497


C:\Users\User\AppData\Local\Temp\ipykernel_3336\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values


In [8]:
# 첫 번째 배치 데이터 확인
first_batch = train_dataset[0]
seq_x, seq_y, seq_x_mark, seq_y_mark = first_batch

print("\n첫 번째 배치 데이터 shape:")
print(f"seq_x shape: {seq_x.shape}")  # 입력 시퀀스
print(f"seq_y shape: {seq_y.shape}")  # 타겟 시퀀스
print(f"seq_x_mark shape: {seq_x_mark.shape}")  # 입력 시퀀스의 시간 특성
print(f"seq_y_mark shape: {seq_y_mark.shape}")  # 타겟 시퀀스의 시간 특성


첫 번째 배치 데이터 shape:
seq_x shape: (96, 7)
seq_y shape: (72, 7)
seq_x_mark shape: (96, 5)
seq_y_mark shape: (72, 5)


In [11]:
seq_x

array([[-0.37081759, -0.01423578, -0.63614802, -0.15291035,  1.34380139,
         0.86776962,  1.46146281],
       [-0.38230376,  0.01782703, -0.65553544, -0.17161981,  1.40242388,
         0.96422113,  1.45372262],
       [-0.38230376, -0.04629865, -0.65553544, -0.18980956,  1.37359334,
         0.82033431,  1.40771833],
       [-0.38230376, -0.04629865, -0.65553544, -0.17161981,  1.37359334,
         0.82033431,  1.07794748],
       [-0.39379001,  0.01782703, -0.65553544, -0.17161981,  1.2851789 ,
         0.916786  ,  1.16232524],
       [-0.42824853, -0.04629865, -0.66187711, -0.18980956,  1.2563479 ,
         0.77131812,  1.154694  ],
       [-0.45122096, -0.07836151, -0.68126453, -0.20851902,  1.08047997,
         0.86776962,  1.14695402],
       [-0.48567948, -0.11042438, -0.68126453, -0.22670876,  0.96323476,
         0.77131812,  1.08568746],
       [-0.48567948, -0.14248719, -0.69412908, -0.20851902,  0.93440399,
         0.67486661,  1.16232524],
       [-0.48567948, -0.1104

In [12]:
seq_y

array([[-0.56591135,  0.24226704, -0.59755436,  0.05029507,  0.02719499,
         1.15712414,  0.22675697],
       [-0.56591135,  0.27432985, -0.57816694,  0.06848483, -0.06121945,
         1.20455927,  0.06563248],
       [-0.56591135,  0.30639266, -0.58468981,  0.05029507,  0.05602576,
         1.25357565,  0.16538146],
       [-0.56591135,  0.30639266, -0.58468981,  0.05029507,  0.08581771,
         1.35002716,  0.15001024],
       [-0.56591135,  0.30639266, -0.60389605,  0.2535005 ,  0.29051641,
         1.30101078,  0.15775043],
       [-0.49716565,  0.43464413, -0.5395733 ,  0.21660132,  0.17327097,
         1.35002716,  0.2803925 ],
       [-0.48567948,  0.46670694, -0.50732134,  0.2535005 ,  0.02719499,
         1.30101078,  0.26502128],
       [-0.48567948,  0.49876986, -0.49445677,  0.27169029, -0.09005022,
         1.15712414,  0.29576351],
       [-0.49716565,  0.43464413, -0.48159224,  0.30858948, -0.23612619,
         1.01323751,  0.26502128],
       [-0.47419331,  0.4987

In [13]:
seq_x_mark

array([[ 7,  1,  4,  0,  0],
       [ 7,  1,  4,  0,  1],
       [ 7,  1,  4,  0,  2],
       [ 7,  1,  4,  0,  3],
       [ 7,  1,  4,  1,  0],
       [ 7,  1,  4,  1,  1],
       [ 7,  1,  4,  1,  2],
       [ 7,  1,  4,  1,  3],
       [ 7,  1,  4,  2,  0],
       [ 7,  1,  4,  2,  1],
       [ 7,  1,  4,  2,  2],
       [ 7,  1,  4,  2,  3],
       [ 7,  1,  4,  3,  0],
       [ 7,  1,  4,  3,  1],
       [ 7,  1,  4,  3,  2],
       [ 7,  1,  4,  3,  3],
       [ 7,  1,  4,  4,  0],
       [ 7,  1,  4,  4,  1],
       [ 7,  1,  4,  4,  2],
       [ 7,  1,  4,  4,  3],
       [ 7,  1,  4,  5,  0],
       [ 7,  1,  4,  5,  1],
       [ 7,  1,  4,  5,  2],
       [ 7,  1,  4,  5,  3],
       [ 7,  1,  4,  6,  0],
       [ 7,  1,  4,  6,  1],
       [ 7,  1,  4,  6,  2],
       [ 7,  1,  4,  6,  3],
       [ 7,  1,  4,  7,  0],
       [ 7,  1,  4,  7,  1],
       [ 7,  1,  4,  7,  2],
       [ 7,  1,  4,  7,  3],
       [ 7,  1,  4,  8,  0],
       [ 7,  1,  4,  8,  1],
       [ 7,  1

In [14]:
seq_y_mark

array([[ 7,  1,  4, 12,  0],
       [ 7,  1,  4, 12,  1],
       [ 7,  1,  4, 12,  2],
       [ 7,  1,  4, 12,  3],
       [ 7,  1,  4, 13,  0],
       [ 7,  1,  4, 13,  1],
       [ 7,  1,  4, 13,  2],
       [ 7,  1,  4, 13,  3],
       [ 7,  1,  4, 14,  0],
       [ 7,  1,  4, 14,  1],
       [ 7,  1,  4, 14,  2],
       [ 7,  1,  4, 14,  3],
       [ 7,  1,  4, 15,  0],
       [ 7,  1,  4, 15,  1],
       [ 7,  1,  4, 15,  2],
       [ 7,  1,  4, 15,  3],
       [ 7,  1,  4, 16,  0],
       [ 7,  1,  4, 16,  1],
       [ 7,  1,  4, 16,  2],
       [ 7,  1,  4, 16,  3],
       [ 7,  1,  4, 17,  0],
       [ 7,  1,  4, 17,  1],
       [ 7,  1,  4, 17,  2],
       [ 7,  1,  4, 17,  3],
       [ 7,  1,  4, 18,  0],
       [ 7,  1,  4, 18,  1],
       [ 7,  1,  4, 18,  2],
       [ 7,  1,  4, 18,  3],
       [ 7,  1,  4, 19,  0],
       [ 7,  1,  4, 19,  1],
       [ 7,  1,  4, 19,  2],
       [ 7,  1,  4, 19,  3],
       [ 7,  1,  4, 20,  0],
       [ 7,  1,  4, 20,  1],
       [ 7,  1

In [15]:
# DataLoader를 사용한 배치 데이터 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)
# 첫 번째 배치 확인
for batch in train_loader:
    batch_x, batch_y, batch_x_mark, batch_y_mark = batch
    print("\nDataLoader의 첫 번째 배치 shape:")
    print(f"batch_x shape: {batch_x.shape}")
    print(f"batch_y shape: {batch_y.shape}")
    print(f"batch_x_mark shape: {batch_x_mark.shape}")
    print(f"batch_y_mark shape: {batch_y_mark.shape}")
    break


DataLoader의 첫 번째 배치 shape:
batch_x shape: torch.Size([32, 96, 7])
batch_y shape: torch.Size([32, 96, 7])
batch_x_mark shape: torch.Size([32, 96, 5])
batch_y_mark shape: torch.Size([32, 96, 5])


In [16]:
# 데이터 경로 설정
root_path = './ETT-small'
data_path = 'ETTm1.csv'

# 원본 데이터 확인
df = pd.read_csv(os.path.join(root_path, data_path))
print("원본 데이터 shape:", df.shape)
print("\n첫 5행 데이터:")
print(df.head())
print("\n컬럼명:", df.columns.tolist())

# Dataset 인스턴스 생성 (scale=False로 설정)
train_dataset = Dataset_ETT_minute(
    root_path=root_path,
    data_path=data_path,
    flag='train',
    size=[96, 48, 48],  
    features='MS',  
    target='OT',
    scale=False  # StandardScaler 적용하지 않음
)

# 첫 번째 배치 데이터 확인
first_batch = train_dataset[0]
seq_x, seq_y, seq_x_mark, seq_y_mark = first_batch

print("\n첫 번째 배치 데이터:")
print("입력 시퀀스(seq_x) 첫 5행:")
print(seq_x[:5])
print("\n타겟 시퀀스(seq_y) 첫 5행:")
print(seq_y[:5])

# 시간 특성 확인
print("\n시간 특성(seq_x_mark) 첫 5행:")
print(seq_x_mark[:5])
print("\n시간 특성 컬럼:", ['month', 'day', 'weekday', 'hour', 'minute'])

# 데이터 범위 확인
print("\n데이터 범위:")
print("seq_x 범위:", np.min(seq_x), "~", np.max(seq_x))
print("seq_y 범위:", np.min(seq_y), "~", np.max(seq_y))

# 각 변수별 통계량 확인
print("\n입력 시퀀스 기술통계:")
seq_x_df = pd.DataFrame(seq_x)
print(seq_x_df.describe())

원본 데이터 shape: (69680, 8)

첫 5행 데이터:
                  date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0  2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1  2016-07-01 00:15:00  5.760  2.076  1.492  0.426  4.264  1.401  30.459999
2  2016-07-01 00:30:00  5.760  1.942  1.492  0.391  4.234  1.310  30.038000
3  2016-07-01 00:45:00  5.760  1.942  1.492  0.426  4.234  1.310  27.013000
4  2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001

컬럼명: ['date', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']

첫 번째 배치 데이터:
입력 시퀀스(seq_x) 첫 5행:
[[ 5.82700014  2.00900006  1.59899998  0.46200001  4.20300007  1.34000003
  30.53100014]
 [ 5.76000023  2.07599998  1.49199998  0.426       4.26399994  1.40100002
  30.45999908]
 [ 5.76000023  1.94200003  1.49199998  0.391       4.23400021  1.30999994
  30.03800011]
 [ 5.76000023  1.94200003  1.49199998  0.426       4.23400021  1.30999994
  27.01300049]
 [ 5.69299984  2.07599998  1.49199998  0.426     

C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values


# val, test set의 시작시점 확인

In [3]:
# 데이터 경로 설정
root_path = './ETT-small'
data_path = 'ETTm1.csv'

# train, validation, test 데이터셋 생성
datasets = {
    'train': Dataset_ETT_minute(
        root_path=root_path,
        data_path=data_path,
        flag='train',
        size=None,
        features='M',
        scale=False
    ),
    'val': Dataset_ETT_minute(
        root_path=root_path,
        data_path=data_path,
        flag='val',
        size=None,
        features='M',
        scale=False
    ),
    'test': Dataset_ETT_minute(
        root_path=root_path,
        data_path=data_path,
        flag='test',
        size=None,
        features='M',
        scale=False
    )
}

# 각 데이터셋의 시작 시점 확인
for name, dataset in datasets.items():
    first_batch = dataset[0]
    _, _, seq_x_mark, _ = first_batch
    
    # 원본 데이터에서 시간 정보 가져오기
    df_raw = pd.read_csv(os.path.join(root_path, data_path))
    
    # border1 계산
    if name == 'train':
        border1 = 0
    elif name == 'val':
        border1 = 12 * 30 * 24 * 4 - dataset.seq_len
    else:  # test
        border1 = 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - dataset.seq_len
    
    # 해당 시점의 실제 날짜 확인
    start_date = pd.to_datetime(df_raw['date'].iloc[border1])
    
    print(f"\n{name.upper()} 데이터셋:")
    print(f"시작 인덱스: {border1}")
    print(f"시작 날짜: {start_date}")
    print("시간 특성 첫 5행 (month, day, weekday, hour, minute):")
    print(seq_x_mark[:5])
    
    # 시작과 끝 시점의 전체 정보
    print(f"\n{name.upper()} 첫 번째 데이터 시점:")
    first_mark = seq_x_mark[0]
    print(f"월: {int(first_mark[0])}, 일: {int(first_mark[1])}, "
          f"요일: {int(first_mark[2])}, 시: {int(first_mark[3])}, "
          f"15분단위: {int(first_mark[4])}")

# 전체 데이터 길이 확인
print("\n전체 데이터 길이:", len(df_raw))
print("각 데이터셋 길이:")
for name, dataset in datasets.items():
    print(f"{name}: {len(dataset)}")

C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values
C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values



TRAIN 데이터셋:
시작 인덱스: 0
시작 날짜: 2016-07-01 00:00:00
시간 특성 첫 5행 (month, day, weekday, hour, minute):
[[7 1 4 0 0]
 [7 1 4 0 1]
 [7 1 4 0 2]
 [7 1 4 0 3]
 [7 1 4 1 0]]

TRAIN 첫 번째 데이터 시점:
월: 7, 일: 1, 요일: 4, 시: 0, 15분단위: 0

VAL 데이터셋:
시작 인덱스: 34176
시작 날짜: 2017-06-22 00:00:00
시간 특성 첫 5행 (month, day, weekday, hour, minute):
[[ 6 22  3  0  0]
 [ 6 22  3  0  1]
 [ 6 22  3  0  2]
 [ 6 22  3  0  3]
 [ 6 22  3  1  0]]

VAL 첫 번째 데이터 시점:
월: 6, 일: 22, 요일: 3, 시: 0, 15분단위: 0

TEST 데이터셋:
시작 인덱스: 45696
시작 날짜: 2017-10-20 00:00:00
시간 특성 첫 5행 (month, day, weekday, hour, minute):
[[10 20  4  0  0]
 [10 20  4  0  1]
 [10 20  4  0  2]
 [10 20  4  0  3]
 [10 20  4  1  0]]

TEST 첫 번째 데이터 시점:
월: 10, 일: 20, 요일: 4, 시: 0, 15분단위: 0

전체 데이터 길이: 69680
각 데이터셋 길이:
train: 34081
val: 11425
test: 11425


C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values


In [5]:
# 데이터 경로 설정
root_path = './ETT-small'
data_path = 'ETTm1.csv'

# 원본 데이터 크기 확인
df = pd.read_csv(os.path.join(root_path, data_path))
print("전체 데이터 크기:", len(df))

# 경계값 계산
seq_len = 24 * 4 * 4    # 384
label_len = 24 * 4      # 96
pred_len = 24 * 4       # 96

border1s = [0, 12 * 30 * 24 * 4 - seq_len, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - seq_len]
border2s = [12 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]

print("\n각 분할의 경계값:")
splits = ['train', 'validation', 'test']
for i, split in enumerate(splits):
    print(f"\n{split}:")
    print(f"시작 인덱스 (border1): {border1s[i]}")
    print(f"종료 인덱스 (border2): {border2s[i]}")
    print(f"데이터 길이: {border2s[i] - border1s[i]}")

# 이론적인 분할 크기
print("\n이론적인 분할:")
total_minutes = 12 * 30 * 24 * 4  # 1년치
print(f"1년: {total_minutes}")
print(f"4개월: {4 * 30 * 24 * 4}")

# 실제 데이터셋 크기
datasets = {
    'train': Dataset_ETT_minute(
        root_path=root_path,
        data_path=data_path,
        flag='train',
        size=None,
        features='M',
        scale=False
    ),
    'val': Dataset_ETT_minute(
        root_path=root_path,
        data_path=data_path,
        flag='val',
        size=None,
        features='M',
        scale=False
    ),
    'test': Dataset_ETT_minute(
        root_path=root_path,
        data_path=data_path,
        flag='test',
        size=None,
        features='M',
        scale=False
    )
}

print("\n실제 데이터셋 크기:")
for name, dataset in datasets.items():
    print(f"{name}: {len(dataset)}")

전체 데이터 크기: 69680

각 분할의 경계값:

train:
시작 인덱스 (border1): 0
종료 인덱스 (border2): 34560
데이터 길이: 34560

validation:
시작 인덱스 (border1): 34176
종료 인덱스 (border2): 46080
데이터 길이: 11904

test:
시작 인덱스 (border1): 45696
종료 인덱스 (border2): 57600
데이터 길이: 11904

이론적인 분할:
1년: 34560
4개월: 11520


C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values
C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values



실제 데이터셋 크기:
train: 34081
val: 11425
test: 11425


C:\Users\User\AppData\Local\Temp\ipykernel_26276\2425535225.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values


# Dataset_Custom으로 한번 해보자 